In [7]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy.special import gamma, gammaincc
from scipy.optimize import fsolve
from ipywidgets import interactive,interact, HBox, Layout,VBox
import ipywidgets as widgets

## All 12 parameters manually specified without constraints

In [2]:
#"constants"
D = 1.66 #Diffusivity factor
sigma = 5.67e-8

In [3]:
n = 1.1 #opacity has slightly positive dependence on pressure
ga = 1.4 #for N2-O2 mixture (N=5)
a = 0.6 #moist/dry
bn = 4*(a*(ga-1)/ga)/n
F2 = 238 #Wm^-2, (1-0.3)*1361/4
F1 = 0.05*F2 #Wm^-2, assume UV radiation is 5% of F2, from wikipedia
Fi = 0.09 #Wm^-2, from wikipedia
k1 = 7000 #(0.001/9.8)*kappa UV = 1, kappaUV/kappa longwave
k2 = 0.6 #kappa shortwave from trenberth diagram (transmittance 184/341 Wm^-2)/kappa longwave
taurc = 2/3
tau0 = 1.5
p0 = 1 #bar
T0 = 288 #K
#prc = 0.1 #typical tropopause pressure
prc=(taurc/tau0)**(1/n)

In [4]:
def Erad(p, p0, n, F1, F2, Fi, k1, k2, tau0):
    tau = tau0*(p/p0)**n
    return ((((F1/2)*(1+D/k1+(k1/D-D/k1)*np.exp(-k1*tau))+
            (F2/2)*(1+D/k2+(k2/D-D/k2)*np.exp(-k2*tau))+
            (Fi/2)*(1+D*tau))/sigma)**(1/4))

def Econv(p, p0, T0, n, ga, a):
    return T0*(p/p0)**((a*(ga-1))/ga)

In [5]:
def f(p0, T0, n, ga, a, F1, F2, Fi, k1, k2, tau0, taurc):
    plt.figure(figsize=(8,6))
    ps = np.linspace(0.001,1.0,100)
    prc=(taurc/tau0)**(1/n)
    
    Ts=[]
    for p in ps:
        if p <= prc:
            Ts.append(Erad(p, p0, n, F1, F2, Fi, k1, k2, tau0))
        else:
            Ts.append(Econv(p, p0, T0, n, ga, a))
    
    plt.semilogy(Ts, ps,'--')

    plt.plot([100, 1000],[prc,prc],'k:',label="Tropopause")
    plt.plot([100, 1000],[.001,.001],'k--',label="Stratopause")

    plt.xlim(100, 1000)
    plt.ylim(1.0, 0.0001)
    plt.xlabel("Temperature [K]",fontsize=12)
    plt.ylabel("Pressure [bar]",fontsize=12)

    #Formatting
    plt.tick_params(top=True, right=True, direction='in', which='both')
    plt.legend(fontsize=12)

    plt.show()
    
#creating sliders corresponding to each f arguments

interactive_plot = interactive(f, 
                               p0=(0.5,2), 
                               T0=(200.,300.),
                               n=(1.,2.),
                               ga=(1.,2.),
                               a=(0.6,0.9,.01),
                               F1=(0.05*200,0.05*1000),
                               F2=(200.,1000.),
                               Fi=(0.01,0.1,.01),
                               k1=(1000,8000,1),
                               k2=(0.1,0.9),
                               tau0=(1.,2.),
                               taurc=(1/3,1))

#specifying slider names
interactive_plot.children[0].description=r'$p_0$'
interactive_plot.children[1].description=r'$T_0$'
interactive_plot.children[2].description=r'$n$'
interactive_plot.children[3].description=r'$\gamma$'
interactive_plot.children[4].description=r'$\alpha$'
interactive_plot.children[5].description=r'$F_1$'
interactive_plot.children[6].description=r'$F_2$'
interactive_plot.children[7].description=r'$F_i$'
interactive_plot.children[8].description=r'$k_1$'
interactive_plot.children[9].description=r'$k_2$'
interactive_plot.children[10].description=r'$\tau_0$'
interactive_plot.children[11].description=r'$\tau_\mathrm{rc}$'

controls = HBox(interactive_plot.children[:-1], layout = Layout(flex_flow='row wrap'))
output = interactive_plot.children[-1]

In [6]:
display(VBox([controls, output]))

## System of equations

### Thermal fluxes at RC boundary

Equation (13) in Robinson & Catling (2012) reads: $$F^+(\tau)=\sigma T_0^4e^{D\tau}\left[e^{-D\tau_0}+\frac{1}{(D\tau_0)^{4\beta/n}}\left(\Gamma\left(1+\frac{4\beta}{n},D\tau\right)-\Gamma\left(1+\frac{4\beta}{n},D\tau_0\right)\right)\right],$$ where the incomplete Gamma function as defined in Appendix is:$$\Gamma(a,x)\equiv \int_x^{\infty}t^{a-1}e^{-t}dt,$$ whereas scipy.special.gammaincc(a,x) computes:$$Q(a,x)=\frac{1}{\Gamma (a)}\int_x^{\infty}t^{a-1}e^{-t}dt\Rightarrow \Gamma(a,x)=Q(a,x)\Gamma (a);$$

which, at $\tau=\tau_\mathrm{rc}$, must agree with (equation 19):$$F^+(\tau)=\frac{F_1}{2}\left[1+\frac{D}{k_1}+\left(1-\frac{D}{k_1}\right)e^{-k_1\tau}\right]+\frac{F_2}{2}\left[1+\frac{D}{k_2}+\left(1-\frac{D}{k_2}\right)e^{-k_2\tau}\right]+\frac{F_i}{2}(2+D\tau).$$

So the first equality is:
\begin{equation}\sigma T_0^4e^{D\tau_\mathrm{rc}}\left[e^{-D\tau_0}+\frac{1}{(D\tau_0)^{4\beta/n}}\left(\Gamma\left(1+\frac{4\beta}{n},D\tau_\mathrm{rc}\right)-\Gamma\left(1+\frac{4\beta}{n},D\tau_0\right)\right)\right]=\frac{F_1}{2}\left[1+\frac{D}{k_1}+\left(1-\frac{D}{k_1}\right)e^{-k_1\tau_\mathrm{rc}}\right]+\frac{F_2}{2}\left[1+\frac{D}{k_2}+\left(1-\frac{D}{k_2}\right)e^{-k_2\tau_\mathrm{rc}}\right]+\frac{F_i}{2}(2+D\tau_\mathrm{rc})
\end{equation}

And the second one (equation 21):$$\sigma T_0^4\left(\frac{\tau_\mathrm{rc}}{\tau_0}\right)^{4\beta/n}=\frac{F_1}{2}\left[1+\frac{D}{k_1}+\left(\frac{k_1}{D}-\frac{D}{k_1}\right)e^{-k_1\tau_\mathrm{rc}}\right]+\frac{F_2}{2}\left[1+\frac{D}{k_2}+\left(\frac{k_2}{D}-\frac{D}{k_2}\right)e^{-k_2\tau_\mathrm{rc}}\right]+\frac{F_i}{2}(2+D\tau_\mathrm{rc})$$

with relevant parameters: $T_0$, $\tau_\mathrm{rc}$, $\tau_0$, $\alpha$, $\gamma$, $n$, $F_1$, $k_1$, $F_2$, $k_2$, $F_i$; and with the relationship $$\tau=\tau_0\left(\frac{p}{p_0}\right)^n.$$

($\tau_0$ and $\tau_\mathrm{rc}$ should be solved and then entered as arguments into f)

In [96]:
def taus(p0, T0, n, ga, a, F1, F2, Fi, k1, k2):
    
    bn = 4*(a*(ga-1)/ga)/n
    
    def equations(t):
        tau0, taurc = t
        return ((F1/2.)*(1+D/k1+(1-D/k1)*sp.exp(-k1*taurc))+\
               (F2/2.)*(1+D/k2+(1-D/k2)*sp.exp(-k2*taurc))+\
               (Fi/2)*(2+D*taurc)-\
               sigma*T0**4*sp.exp(D*taurc)*(sp.exp(-D*tau0)+(1/(D*tau0)**bn)*\
               (gammaincc(1+bn,D*taurc)*gamma(1+bn)-gammaincc(1+bn,D*tau0)*gamma(1+bn))),
               (F1/2.)*(1+D/k1+(k1/D-D/k1)*sp.exp(-k1*taurc))+\
               (F2/2.)*(1+D/k2+(k2/D-D/k2)*sp.exp(-k2*taurc))+\
               (Fi/2)*(2+D*taurc)-\
               sigma*T0**4*(taurc/tau0)**bn)
    
    #tau0, taurc = sp.symbols('tau0, tau')
    
    solutions = fsolve(equations,(1.5,2/3))
    
    return(solutions)

In [97]:
#Titan
taus(1.1, 191., 2., 1.4, 0.85, 1.3, 7., 5.4, 100., 0.06)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


array([10.34030138,  3.89151002])

In [107]:
def ten_params(p0, T0, n, ga, a, F1, F2, Fi, k1, k2):
    p0 = float(p0)
    T0 = float(T0)
    n = float(n)
    ga = float(ga)
    a = float(a)
    F1 = float(F1)
    F2 = float(F2)
    Fi = float(Fi)
    k1 = float(k1)
    k2 = float(k2)
    
    
    plt.figure(figsize=(8,6))
    ps = np.linspace(0.001,1.4,1000)
    
    tau0, taurc = taus(p0, T0, n, ga, a, F1, F2, Fi, k1, k2)
    
    prc=p0*(taurc/tau0)**(1/n)
    
    Ts=[]
    for p in ps:
        if p <= prc:
            Ts.append(Erad(p, p0, n, F1, F2, Fi, k1, k2, tau0))
        else:
            Ts.append(Econv(p, p0, T0, n, ga, a))
    
    plt.semilogy(Ts, ps,'--')

    plt.plot([min(Ts)-20, max(Ts)+20],[prc,prc],'k:',label="Tropopause")
    plt.plot([min(Ts)-20, max(Ts)+20],[.001,.001],'k--',label="Stratopause")

    plt.title(r'$\tau_0=$'+
              str(round(tau0,2))+
              r', $\tau_\mathrm{rc}=$'+
              str(round(taurc,2))+
              r', $p_\mathrm{rc}=$'+
              str(round(prc,2)))
    
    plt.xlim(min(Ts)-20, max(Ts)+20)
    plt.ylim(1.5, 0.0001)
    plt.xlabel("Temperature [K]",fontsize=12)
    plt.ylabel("Pressure [bar]",fontsize=12)

    #Formatting
    plt.tick_params(top=True, right=True, direction='in', which='both')
    plt.legend(fontsize=12)

    plt.show()
    
#creating sliders corresponding to each f arguments

interactive_plot = interactive(ten_params, p0='1.5', T0='94', n='1.33333', ga='1.4', a='0.77',
                               F1='1.5', F2='1.1', Fi='0', k1='120', k2='0.2')

#specifying slider names
interactive_plot.children[0].description=r'$p_0$'
interactive_plot.children[1].description=r'$T_0$'
interactive_plot.children[2].description=r'$n$'
interactive_plot.children[3].description=r'$\gamma$'
interactive_plot.children[4].description=r'$\alpha$'
interactive_plot.children[5].description=r'$F_1$'
interactive_plot.children[6].description=r'$F_2$'
interactive_plot.children[7].description=r'$F_i$'
interactive_plot.children[8].description=r'$k_1$'
interactive_plot.children[9].description=r'$k_2$'

controls = HBox(interactive_plot.children[:-1], layout = Layout(flex_flow='row wrap'))
output = interactive_plot.children[-1]

### Jupiter (model c)

In [99]:
display(VBox([controls, output]))

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

### Titan (model n=4/3)

In [108]:
display(VBox([controls, output]))